<a href="https://colab.research.google.com/github/itruonghai/yolov5/blob/master/Running_Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://user-images.githubusercontent.com/26833433/82952157-51b7db00-9f5d-11ea-8f4b-dda1ffecf992.jpg">

This notebook was written by Ultralytics LLC, and is freely available for redistribution under the [GPL-3.0 license](https://choosealicense.com/licenses/gpl-3.0/). 
For more information please visit https://github.com/ultralytics/yolov5 and https://www.ultralytics.com.

# Setup

Clone repo, install dependencies, `%cd` into `./yolov5` folder and check GPU.

# Download Dataset at: https://drive.google.com/file/d/1zqmCOyjKmGbYtLsdJXkm5RdGxEWW-Etj/view?usp=sharing


In [ ]:
!nvidia-smi 

Thu Nov  5 17:15:34 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.32.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    25W / 300W |      0MiB / 16130MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!git clone https://github.com/itruonghai/yolov5  # clone repo
!pip install wandb
!pip install -r yolov5/requirements.txt  # install dependencies (ignore errors)
%cd yolov5

import torch
from IPython.display import Image, clear_output  # to display images
from utils.google_utils import gdrive_download  # to download models/datasets

clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

Setup complete. Using torch 1.7.0+cu101 _CudaDeviceProperties(name='Tesla V100-SXM2-16GB', major=7, minor=0, total_memory=16130MB, multi_processor_count=80)


In [ ]:
!mkdir dataset/labels
!mkdir dataset/labels/train
!mkdir dataset/labels/val
!mkdir dataset/labels/test



In [ ]:
import json
import os  
def split(mode):
    with open('dataset/annotations/{}.json'.format(mode)) as f:
        data = json.load(f)
    name = os.listdir('dataset/images/{}'.format(mode))
    annotations = data['annotations']
    for annotation in annotations:
        if '{}.png'.format(annotation['image_id']) in name:
            with open('dataset/labels/{}/{}.txt'.format(mode,annotation['image_id']), 'a') as f:
                x,y,w,h = annotation['bbox']
                x_center = (x + w/2) / 1622 
                y_center = (y + h/2) / 626 
                w = w / 1622
                h = h / 626 
                f.write('{} {} {} {} {}\n'.format(annotation['category_id']-1 , x_center, y_center, w, h))
for mode in ['train', 'val', 'test']:   
    split(mode)


# Train 300 epochs


In [ ]:
!python train.py --img-size 1024 --batch 16 --epochs 300 --data coco128.yaml --weights '' --cfg yolov5x.yaml  --cache